In [2]:
import numpy as np
import pandas as pd
import requests
import bs4
import re
import sys
import logging
import urllib
import re
import json
import sys




In [9]:

BASEURL='https://openapi.naver.com/v1/search/movie.json?query=%s'
DISPLAY=BASEURL+'&display=%s'
START=DISPLAY+'&start=%s'

client_id='mGheSDZSqWyhMF5gFAqU'
client_secret='i3HQTMe9Sx'

def collectMovie(page_number=10):
    assert page_number<20001,'page number must be below 20000'
    movies=[]
    for page in range(1,page_number):
        url='https://movie.naver.com/movie/point/af/list.nhn?&page=%s' %page
        res=requests.get(url.format(1))
        obj=bs4.BeautifulSoup(res.text)
        cnts=obj.select('tbody > tr')
        for cnt in cnts:
            movie=cnt.find('a',class_='movie color_b').contents[0]
            movies.append(movie)
    return movies

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def getUrl(title,display=None,start=None):
    #The internal system will recognize foregin languages  as special charcaters,which leads to UnicoderError
    #unless the are correctly quoted. 
    title=urllib.parse.quote(title)
    
    if (display !=None and start ==None):
        myurl=DISPLAY %(title,display)
    elif (display !=None and start !=None):
        myurl=START %(title,display,start)
    else:
        myurl=BASEURL %(title)
    
    return myurl


def findItem(items,index,score_results):
    if len(items)==0:
        print("No Result")
        sys.exit(1)
    else:
        for item in items:
            link=item['link']
            result=re.split('code=',link)[1]
            score_results[index]={'title_KOR':cleanhtml(item['title']),'title_ENG':cleanhtml(item['subtitle']),'link_code':result\
                                  ,'user_rating':item['userRating']}
            index+=1
    return score_results,index
    

def searchByTitle(titles,display=2,start=1):
    assert type(display)==int,'display must be integer'
    assert type(start)==int,'start must be integer'
    score_results={}
    index=0
    for title in titles:
        myurl=getUrl(title,display,start)
        header_params = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}
        response=requests.get(myurl,headers=header_params)
        if response.status_code==200:
            data=json.loads(response.text)['items']
            result,index=findItem(data,index,score_results)
        else:
            pass
    return result

genres={'드라마':'Drama','판타지':'Fantasy','서부':'Western','공포':'Horror','로맨스':'Romance'\
        ,'모험':'Adventure','스릴러':'Thriller','느와르':'Noir','컬트':'Cult' ,'다큐멘터리':'Documentary'\
        ,'코미디':'Comedy','가족':'Family','미스터리':'Mystery',"전쟁":'War','애니메이션':'Animation'\
        ,'범죄':'Crime','뮤지컬':'Musical','SF':'Science Fiction','액션':'Action','무협':'Martial Art'\
        ,'에로':'Erotic Movies','서스펜스':'Suspense','서사':'Epic','블랙코미디':'Black Comedy'\
        ,'실험':'Experimental Film','영화카툰':'Animated Movies','영화음악':'Musical film','영화패러디포스터':'paradies'
        ,'멜로/로맨스':'Romance'}

countries={'한국':'KOR','일본':'JAP','미국':'US','홍콩':'HK','영국':'GB','프랑스':'FR'}


def getFull(codes):
    BASEURL='https://movie.naver.com/movie/bi/mi/basic.nhn?code=%s'
    count=0
    score_results=dict()
    for code in codes:
        url=BASEURL %code
        res=requests.get(url.format(1))
        obj=bs4.BeautifulSoup(res.text)
        try:
            specs=obj.find('dl',class_='info_spec').select('dd > p > span > a')
            types=[]
            nations=[]
            for spec in specs:
                if re.search('[^0-9. ]+',spec.text): 
                    if spec.text in genres:
                        types.append(genres[spec.text])
                    elif spec.text in countries:
                        nations.append(countries[spec.text])

            types=','.join(types)
            nations=','.join(nations)
        except:
            types='Not Specified'
            nations='Not Specified'
        
        try:
            critics_score=obj.find_all('span',class_='st_off')[1]
            critic=re.search('[0-9.%]+',str(critics_score)).group()
        except:
            ciritc=np.nan
        
        
        #just in case where no comments are avaialbe we should ignore the movie
        try:
            info=obj.find_all('div',class_='score_result')[0].find_all('li')
            for li in info:
                score_results[count]={'score':int(li.em.text),'text':li.p.text.strip()\
                          ,'critics_score':critic,'link_code':code,'genres':types,'nations':nations}
                count+=1
        except:
            print('no comments exist for this movie')
            pass
        
           
    return score_results



if __name__=='__main__':
#     #collect movie titles
#     titles=collectMovie(page_number=20000)
#     titles=list(titles)
#     data=searchByTitle(titles)
#     data=pd.DataFrame(data).T
    #data.to_csv(r'C:\Users\sd4be\Desktop\data\naver\data1.csv')
    data=pd.read_csv(r'C:\Users\sd4be\Desktop\data\naver\data1.csv')
    codes=data.link_code
    #eliminate duplicate movide codes
    codes=codes.unique()
    comment_data=getFull(codes)
    comment_data=pd.DataFrame(comment_data).T
    comment_data.to_csv(r'C:\Users\sd4be\Desktop\data\naver\data2.csv')


C:\Users\sd4be\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\sd4be\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comments exist for this movie
no comment

In [239]:
comment_data

,score,text,critics_score,link_code,genres,nations
0,10,나이가 들고 어엿한 성인이 된 해리포터는 마법지팡이를 내려두고 두손에 권총을 드는데...,50.0%,188993,Action,GB
1,9,충분함. 가뭄난 극장에 단비 같은 영화다.,50.0%,188993,Action,GB
2,8,화려한 액션 장면이 가장 큰 볼거리로 청불인 만큼 유혈이 낭자하는데 마치 게임처럼 ...,50.0%,188993,Action,GB
3,8,무자비한 청불급 연출과 스토리의 개연성이 당황스럽지만 오랜만에 영화관에 개봉한 신작...,50.0%,188993,Action,GB
4,8,해리포터 근황이 궁금하면 강추 킬링타임용으로 보기 만족 액션도 괜찮음,50.0%,188993,Action,GB
...,...,...,...,...,...,...
96349,8,"김민희는 배우로서 정말 매력적인데, 그 매력을 편안한 마음으로 감상할 수 없음이 안...",70.0%,159827,Drama,"FR,KOR"
96350,9,스크린안의 김민희가 좋다. 꿈을 꾸게 만드는 매력.. 누가 뭐래도 나는 배우 김민희...,70.0%,159827,Drama,"FR,KOR"
96351,6,김민희 연기는 훌륭하지만. 만희한테 숏팬츠 입었다고 화내는 장면은 최악. 아가씨에서...,70.0%,159827,Drama,"FR,KOR"
96352,1,김민희라는 배우가 홍상수라는 감독의 영화에만 출연하는 상황이 너무. 짜증난. 매력이...,70.0%,159827,Drama,"FR,KOR"
